In [1]:
import pygame,math,random
from pygame import mixer

#Interface - Main menu, gameplay, sprites

# Import pygame.locals for easier access to key coordinates
# Updated to conform to flake8 and black standards
from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

class Button():
	def __init__(self, image, pos, text_input, font, base_color, hovering_color):
		self.image = image
		self.x_pos = pos[0]
		self.y_pos = pos[1]
		self.font = font
		self.base_color, self.hovering_color = base_color, hovering_color
		self.text_input = text_input
		self.text = self.font.render(self.text_input, True, self.base_color)
		if self.image is None:
			self.image = self.text
		self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))
		self.text_rect = self.text.get_rect(center=(self.x_pos, self.y_pos))

	def update(self, screen):
		if self.image is not None:
			screen.blit(self.image, self.rect)
		screen.blit(self.text, self.text_rect)

	def checkForInput(self, position):
		if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
			return True
		return False

	def changeColor(self, position):
		if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
			self.text = self.font.render(self.text_input, True, self.hovering_color)
		else:
			self.text = self.font.render(self.text_input, True, self.base_color)   

pygame.init()


SCREEN = pygame.display.set_mode((1280, 720))
pygame.display.set_caption("TopDownShootingGame")

BG = pygame.image.load("bgm3.png")

#To hide un-needed images by blitting with alpha=0
s = pygame.Surface((1280,720))  
s.set_alpha(128)                # alpha level
s.fill((0,0,0))         
SCREEN.blit(s, (0,0))    

#Backgroud music
#mixer.music.load('bgm.wav')
#mixer.music.play(-1)

def get_font(size): # Returns Press-Start-2P in the desired size
    return pygame.font.Font("Font.ttf", size)

def get_font2(size): # Returns Press-Start-2P in the desired size
    return pygame.font.Font("Canada.ttf", size)

def play():
     while True:
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()  #Initialize variables:
        
        clock = pygame.time.Clock()
        screen_width = 850
        screen_height = 600
        surface = pygame.display.set_mode((screen_width,screen_height))
        green = 0,255,0,0
        red = 255,0,0
        blue = 0,0,255
        yellow = 255,255,0
        white = 255,255,255
        black = 0,0,0
        transparent = pygame.Color(0,0,0,255)

        #Background Map
        bg = pygame.image.load("b2.png")

        pygame.display.set_caption("Top Down Shooter")
        icon = pygame.image.load('icon_1.jpg')
        pygame.display.set_icon(icon)

        #Enemy
        enemy_sprites = [pygame.image.load('e1.png'),pygame.image.load('e2.png'),pygame.image.load('e3.png'),pygame.image.load('e4.png'),
                        pygame.image.load('e5.png'),pygame.image.load('e6.png'),pygame.image.load('e7.png'),pygame.image.load('e8.png'),
                        pygame.image.load('e9.png'),pygame.image.load('e10.png'),
                        pygame.image.load('e11.png'),pygame.image.load('e12.png'),pygame.image.load('e13.png'),pygame.image.load('e14.png'),
                        pygame.image.load('e15.png'),pygame.image.load('e16.png'),pygame.image.load('e17.png'),pygame.image.load('e18.png'),
                        pygame.image.load('e19.png'),pygame.image.load('e20.png')]

        # Score
        score_value = 0
        font = pygame.font.Font('freesansbold.ttf', 32)

        textX = 10
        testY = 10

        # Game Over
        over_font = pygame.font.Font('freesansbold.ttf', 64)



        class Square:
            def __init__(self, color, x, y, width, height, speed):
                self.rect = pygame.Rect(x,y,width,height)
                self.color = color
                self.direction = 'E'
                self.speed = speed

            def move(self):
                if self.direction == 'E':
                    self.rect.x = self.rect.x+self.speed
                if self.direction == 'W':
                    self.rect.x = self.rect.x-self.speed
                if self.direction == 'N':
                    self.rect.y = self.rect.y-self.speed
                if self.direction == 'S':
                    self.rect.y = self.rect.y+self.speed

            def moveDirection(self, direction):
                if direction == 'E':
                    self.rect.x = self.rect.x+self.speed
                if direction == 'W':
                    self.rect.x = self.rect.x-self.speed
                if direction == 'N':
                    self.rect.y = self.rect.y-self.speed
                if direction == 'S':
                    self.rect.y = self.rect.y+self.speed

            def collided(self, other_rect):
                #Return True if self collided with other_rect
                return self.rect.colliderect(other_rect)

            def draw(self, surface):
                pygame.draw.rect(s, self.color, self.rect)

            def img(self, image):
                surface.blit(image,(self.rect.x,self.rect.y))

            def move_towards_player(self, player):
                # Find direction vector (dx, dy) between enemy and player.
                dx, dy = player.rect.x - self.rect.x, player.rect.y - self.rect.y
                dist = math.hypot(dx, dy)
                if dist>0:
                    dx, dy = dx / dist, dy / dist  # Normalize.
                    # Move along this normalized vector towards the player at current speed.
                    self.rect.x += dx * self.speed
                    self.rect.y += dy * self.speed

            def choose_sprite(self):
                enemyImg = random.choice(enemy_sprites)
                enemyImg = pygame.transform.scale(enemyImg, (90,90))
                #enemy_rect = enemyImg.get_rect()
                enemy_rect = enemyImg.get_rect(center=(self.rect.x+36,self.rect.y+36))   
                #pygame.draw.rect(surface, (255,0,0), enemy_rect)
                surface.blit(enemyImg,enemy_rect)


        #Inheritance
        class Bullet(Square):
            def __init__(self, color, x, y, width, height, speed, targetx,targety):
                super().__init__(color, x, y, width, height, speed)
                angle = math.atan2(targety-y, targetx-x) #get angle to target in radians
                print('Angle in degrees:', int(angle*180/math.pi))
                self.dx = math.cos(angle)*speed
                self.dy = math.sin(angle)*speed
                self.x = x
                self.y = y
            #Override
            def move(self):
                #self.x and self.y are floats (decimals) so I get more accuracy
                #if I change self.x and y and then convert to an integer for
                #the rectangle.
                self.x = self.x + self.dx
                self.y = self.y + self.dy
                self.rect.x = int(self.x)
                self.rect.y = int(self.y)





        #Build a square
        sq = Square(green,200,200,100,100, 10)

        #Player
        playerImg = pygame.image.load('pl_1 - Copy.png')
        playerImg = pygame.transform.scale(playerImg, (150,150))

        #Player
        bulletImg = pygame.image.load('cannon_3.png')
        bulletImg = pygame.transform.scale(bulletImg, (25,25))

        enemyImg = pygame.image.load('e11.png')
        enemyImg = pygame.transform.scale(enemyImg, (120,120))
        enemy_rect = enemyImg.get_rect()


        s = pygame.Surface((850,600))  # the size of your rect
        s.set_alpha(128)                # alpha level
        s.fill((0,0,0))           # this fills the entire surface
        surface.blit(s, (0,0))    # (0,0) are the top-left coordinates

        bullets = []
        enemies = []

        def show_score(x, y):
            score = font.render("Score : " + str(score_value), True, (255, 255, 255))
            surface.blit(score, (x, y))

        def game_over_text():
            over_text = over_font.render("GAME OVER", True, (255, 255, 255))
            surface.blit(over_text, (225, 250))

        gameover=0

        #Main program loop
        running = False
        while not running:
            surface.fill((0,0,0))
            surface.blit(bg,(0,0))

            #Get user input
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = True
                elif event.type == pygame.KEYDOWN:
                    print(event.key) #Print value of key press
                    """if event.key==119: #w
                        sq.direction = 'N'
                    if event.key==97: #a
                        sq.direction = 'W'
                    if event.key==115: #s
                        sq.direction = 'S'
                    if event.key==100: #d
                        sq.direction = 'E'"""
                    if event.key==32: #spacebar
                        #Fire a bullet
                        spawnx = sq.rect.x + sq.rect.width/2 - 10
                        b = Square(red, spawnx,sq.rect.y, 5,5, 20)
                        b.direction = 'N'
                        bullets.append(b)
                if event.type == pygame.MOUSEBUTTONDOWN:
                    x,y = pygame.mouse.get_pos()
                    #print(x,y)
                    b = Bullet(red, sq.rect.centerx, sq.rect.centery, 10,10, 20, x,y)
                    bullets.append(b)

            #Handle held down keys
            pressed = pygame.key.get_pressed()
            if pressed[pygame.K_w]:
                sq.moveDirection('N')
            if pressed[pygame.K_a]:
                sq.moveDirection('W')
            if pressed[pygame.K_s]:
                sq.moveDirection('S')
            if pressed[pygame.K_d]:
                sq.moveDirection('E')

            #Update game objects
            for b in bullets:
                b.move()
            for e in enemies:
                #e.move()
                e.move_towards_player(sq)

            #spawn enemies on the top of the screen and tell them to move down
            if random.randint(1,30) == 15: #15 doesn't matter
                x = random.randint(0,screen_width-40)
                e = Square(yellow, x,-40, 120,120, 7)
                #e.choose_sprite()
                e.direction = 'S'
                enemies.append(e)
            i=len(enemies)

            #Check for collisions
            '''for b in bullets:
                for e in enemies:
                    if b.collided(e.rect):
                        #e.color = white #TESTING
                        enemies.remove(e)
                        bullets.remove(b)'''
            for i in reversed(range(len(bullets))):
                for j in reversed(range(len(enemies))):
                    if bullets[i].collided(enemies[j].rect):
                    #if bullets[i].collided(enemy_rect):
                        #e.color = white #TESTING
                        score_value+=1
                        del enemies[j]
                        del bullets[i]
                        break
                    if enemies[j].collided(sq.rect):
                        del enemies[0:len(enemies)]
                        gameover=1
                        break            

            if gameover==0:
                pass
            else:
                break

            #Aiming, rotation and drawing

            #Aiming of player
            #player aims in direction of mouse pointer
            correction_angle = 0
            mouse_x, mouse_y = pygame.mouse.get_pos()
            player_rect = playerImg.get_rect(center=(sq.rect.x,sq.rect.y))
            #pygame.draw.rect(surface, (255,0,0), player_rect)

            #getting the vector from player to mouse pointer
            dx, dy = mouse_x - player_rect.centerx-38, player_rect.centery - mouse_y
            angle = math.degrees(math.atan2(dy, dx)) - correction_angle

            #rotating the player
            rot_image = pygame.transform.rotate(playerImg, angle)
            rot_image_rect = rot_image.get_rect(center=player_rect.center)
            surface.blit(rot_image, rot_image_rect.topleft)

            #Aiming of enemy
            #enemy aims in direction of player
            e_correction_angle = 270

            #getting the vector from enemy to player
            edx, edy = player_rect.centerx+38 - enemy_rect.centerx  , enemy_rect.centery - player_rect.centery
            e_angle = math.degrees(math.atan2(edy, edx)) - e_correction_angle

            #rotating the enemy
            e_rot_image = pygame.transform.rotate(enemyImg, e_angle)
            e_rot_image_rect = e_rot_image.get_rect(center=enemy_rect.center)
            #surface.blit(e_rot_image, e_rot_image_rect.topleft)


            #pygame.display.update()
            for b in bullets:
                b.draw(surface)
                bullet_rect = bulletImg.get_rect(center=(b.rect.x,b.rect.y))   
                #pygame.draw.rect(surface, (255,0,0), enemy_rect)
                surface.blit(bulletImg,bullet_rect)
            for e in enemies:
                e.draw(surface)
                #e.choose_sprite()
                enemy_rect = e_rot_image.get_rect(center=(e.rect.x+36,e.rect.y+36))   
                #pygame.draw.rect(surface, (255,0,0), enemy_rect)
                surface.blit(e_rot_image,enemy_rect)

            show_score(textX, testY)
            sq.draw(surface)
            pygame.display.flip()
            clock.tick(30) #30 FPS

        running = False
        while not running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = True
                game_over_text()
            pygame.display.flip()

        pygame.quit()  


        

def credits():
     while True:
            OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
            
           
            
            pygame.display.set_caption('End credits')
            
            bg2_img = pygame.image.load('bg2.png')
            bg2_img = pygame.transform.scale(bg2_img,(1280,720))
            screen=pygame.display.set_mode([1280,720])
            
            
            red = (255,10,0)
            green = (0,255,0)
            blue = (0,0,255)
            darkBlue = (0,0,128)
            white = (255,255,255)
            black = (0,0,0)
            pink = (255,200,200)

            #closing credits or end credits are a list of the movie cast or crew
            movie_credits = '''
            'GAME NAME'
                  
            CREATED BY-
            
            BT21ECE004-PRANAV DESHPANDE
            
            BT21ECE005-KATHAN PATEL
            
            BT21ECE006-SIDDHANT NARKHEDE
            
            
            
            PROGRAM ROLES-
            
            PRANAV: MAIN MENU AND DESIGN
            
            SIDDHANT: PLAYER AND AIMING PHYSICS,SHOOTING AND COLLISON
            
            KATHAN:  ENEMY AND SPAWNING
            '''


            centerx, centery = screen.get_rect().centerx, screen.get_rect().centery
            deltaY = centery + 50  # adjust so it goes below screen start


            running =True
            while running:
                for event in pygame.event.get():
                    if event.type == QUIT:
                        running = False

                screen.fill(0)
                deltaY-= 0.25
                i=0
                msg_list=[]
                pos_list=[]

                font = pygame.font.SysFont("Verdana", 30)

                #msg = font.render('Hello there, how are you?', True, red)`
                for line in movie_credits.split('\n'):
                    msg=font.render(line, True, red)
                    msg_list.append(msg)
                    pos= msg.get_rect(center=(centerx, centery+deltaY+30*i))
                    pos_list.append(pos)
                    i=i+1

                #pos = msg.get_rect(center=(centerx, centery+deltaY))


                #if (centery + deltaY < 0):
                #   running = False         # no repetition - once text scrolls up past screen, over 

                #screen.blit(msg, pos)
                for j in range(i):
                    screen.blit(msg_list[j], pos_list[j])

                pygame.display.update()
                
            pygame.quit()
    



def how_to_play():
    
    
    while True:
         
        mixer.music.load('stranger.mp3')  
        mixer.music.play(-1)  
        
        OPTIONS_MOUSE_POS = pygame.mouse.get_pos()
        
       
      

        SCREEN.fill("white")
        
       
    

        OPTIONS_TEXT = get_font2(50).render("Top Menu Shooter Game", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 50))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)

        OPTIONS_TEXT = get_font(45).render("Welcome!This is a Tutorial on How to Play this game", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 120))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
        OPTIONS_TEXT = get_font(45).render("A.RULES AND INSTRUCTIONS", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(280, 200))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
        OPTIONS_TEXT = get_font(40).render("1.Player has to shoot down the incoming enemies.", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(420, 250))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
        OPTIONS_TEXT = get_font(40).render("2.If the Enemy touches the player,the player is Dead and the Game is Over", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 300))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
                
        OPTIONS_TEXT = get_font(40).render("3.The number of Enemies you shoot down is your score. ", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(470, 350))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
                 
        OPTIONS_TEXT = get_font(45).render("B.CONTROLS ", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(140, 440))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
        
        OPTIONS_TEXT = get_font(45).render("1.Use W-A-S-D to Move the player ", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(340, 490))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
                
        OPTIONS_TEXT = get_font(45).render("2.Use Left Button of the Mouse or Space button to shoot ", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(530, 540))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
        
    
        OPTIONS_TEXT = get_font(50).render("GOODLUCK!", True, "Black")
        OPTIONS_RECT = OPTIONS_TEXT.get_rect(center=(640, 650))
        SCREEN.blit(OPTIONS_TEXT, OPTIONS_RECT)
        
        
        
        
        
        OPTIONS_BACK = Button(image=None, pos=(75, 25), 
                            text_input="BACK", font=get_font(40), base_color="Black", hovering_color="Green")

        OPTIONS_BACK.changeColor(OPTIONS_MOUSE_POS)
        OPTIONS_BACK.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
               
            if event.type == pygame.MOUSEBUTTONDOWN:
                if OPTIONS_BACK.checkForInput(OPTIONS_MOUSE_POS):
                     main_menu()
        
          
      
       
        pygame.display.update()
        
    
    
        
        
def main_menu():
    
   
    while True:
        
        
        
        SCREEN.blit(BG, (0, 0))

        MENU_MOUSE_POS = pygame.mouse.get_pos()
        
        
        MENU_TEXT = get_font2(80).render("MAIN MENU ", True, "#b68f40")  
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 60))
        
        image=pygame.image.load("transparent.png")
        image = pygame.transform.scale(image, (555,150))

        PLAY_BUTTON = Button(image, pos=(640, 180), 
                            text_input="PLAY", font=get_font(75), base_color="#d7fcd4", hovering_color="White")
        GUIDE_BUTTON = Button(image, pos=(640, 335), 
                            text_input="HOW TO PLAY", font=get_font(75), base_color="#d7fcd4", hovering_color="White")
        CREDITS_BUTTON = Button(image, pos=(640, 495), 
                            text_input="CREDITS", font=get_font(75), base_color="#d7fcd4", hovering_color="White")
        QUIT_BUTTON = Button(image, pos=(640, 655), 
                            text_input="QUIT", font=get_font(75), base_color="#d7fcd4", hovering_color="White")

        SCREEN.blit(MENU_TEXT, MENU_RECT)

        for button in [PLAY_BUTTON, GUIDE_BUTTON, CREDITS_BUTTON, QUIT_BUTTON]:
            button.changeColor(MENU_MOUSE_POS)
            button.update(SCREEN)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            
            if event.type == pygame.MOUSEBUTTONDOWN:
                if PLAY_BUTTON.checkForInput(MENU_MOUSE_POS):
                    play()
                if GUIDE_BUTTON.checkForInput(MENU_MOUSE_POS):
                    how_to_play()
                if CREDITS_BUTTON.checkForInput(MENU_MOUSE_POS):
                    credits()
                if QUIT_BUTTON.checkForInput(MENU_MOUSE_POS):  
                    pygame.quit()
                    

        pygame.display.update()
        
#mixer.music.load('bgm.wav')  
#mixer.music.play(-1)
main_menu()        

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: video system not initialized